## Overall Prediction Observation


In [11]:
# Define variables
IS_FILTERED = True
ARCHIVED = None

SHOW_HEADER = True
SHOW_LAST_APPEARANCE = None

In [12]:
# Import libraries
import sys
sys.path.append('../../prediction')

import numpy as np
import matplotlib.pyplot as plt

from putils.observation import compute_rmse, compute_mae, compute_mape, load_data_from_tuned_folder

In [13]:
# Load data
dfs = {
    "l1": load_data_from_tuned_folder("l1", last_appearance=SHOW_LAST_APPEARANCE),
    "l2": load_data_from_tuned_folder("l2", last_appearance=SHOW_LAST_APPEARANCE),
    "l3": load_data_from_tuned_folder("l3", last_appearance=SHOW_LAST_APPEARANCE),
}

# Make L1 and L2 data have the same length with L3 if SHOW_LAST_APPEARANCE = None
if SHOW_LAST_APPEARANCE is None:
    dfs["l1"] = dfs["l1"].tail(len(dfs["l3"]))
    dfs["l2"] = dfs["l2"].tail(len(dfs["l3"]))

# Indicate which columns to ignore
IGNORED_COLS = ["Time", "Actual"] # No need to include "FormattedTime" because it is used as index
if IS_FILTERED: IGNORED_COLS.append("Raw")

In [14]:
# Compute RMSE, MAE, and MAPE for each model
for key, df in dfs.items():
    print("Layer", key)
    for column in df.columns:
        if column not in IGNORED_COLS:
            actual = df["Raw"] if IS_FILTERED else df["Actual"]
            rmse = compute_rmse(actual, df[column])
            mape = compute_mape(actual, df[column])
            mae = compute_mae(actual, df[column])
            print(f"{column} - RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%")
    print("------------------")

Layer l1
ARIMA - RMSE: 1049.1569, MAE: 746.8066, MAPE: 67.70%
SARIMA - RMSE: 1049.1745, MAE: 745.4478, MAPE: 67.46%
ETS - RMSE: 1459.6899, MAE: 1024.6374, MAPE: 94.28%
RNN - RMSE: 642.3348, MAE: 495.1957, MAPE: 49.11%
LSTM - RMSE: 665.7295, MAE: 516.6156, MAPE: 50.66%
CNN - RMSE: 741.9941, MAE: 576.8489, MAPE: 58.00%
GRU - RMSE: 649.1460, MAE: 501.2780, MAPE: 49.63%
TCN - RMSE: 847.4626, MAE: 667.0838, MAPE: 66.95%
------------------
Layer l2
LINEAR_REGRESSION - RMSE: 637.3582, MAE: 489.8354, MAPE: 47.91%
RANDOM_FOREST - RMSE: 649.3304, MAE: 498.4929, MAPE: 48.91%
FEEDFORWARD_NEURAL_NETWORK - RMSE: 650.6872, MAE: 498.2397, MAPE: 48.63%
------------------
Layer l3
Predicted - RMSE: 640.2040, MAE: 491.5452, MAPE: 48.15%
------------------


In [15]:
# # Plot the data
# header = (str(ARCHIVED) if ARCHIVED is not None else "Current") + " - "
# header = header if SHOW_HEADER else ""

# for key, df in dfs.items():
#     # Plot the data
#     fig, axes = plt.subplots(
#         nrows=len(df.columns) - len(IGNORED_COLS),
#         figsize=(10, 6 * (len(df.columns) - len(IGNORED_COLS))),
#     )

#     # Ensure axes is always a list or array
#     if not isinstance(axes, np.ndarray):
#         axes = [axes]

#     ax_idx = 0  # Separate index for accessing axes

#     for column in df.columns:
#         if column not in IGNORED_COLS:
#             actual_label = "Reduced Noise" if IS_FILTERED else "Raw"
#             actual_color = "green" if IS_FILTERED else "orange"
#             axes[ax_idx].plot(df.index, df[column], label=column)
#             if IS_FILTERED:
#                 axes[ax_idx].plot(
#                     df.index,
#                     df["Raw"],
#                     label="Raw",
#                     linestyle="-.",
#                     color="orange",
#                 )  # Added this line to plot the Raw values
#             axes[ax_idx].plot(
#                 df.index,
#                 df["Actual"],
#                 label=actual_label,
#                 linestyle="--",
#                 color=actual_color,
#             )

#             # Sparse Tick Labels logic
#             n = 12  # Display every n-th label
#             for index, label in enumerate(axes[ax_idx].xaxis.get_ticklabels()):
#                 if index % n != 0:
#                     label.set_visible(False)

#             # Rotate the visible x-axis labels
#             axes[ax_idx].tick_params(axis="x", rotation=45)

#             axes[ax_idx].legend()
#             # axes[ax_idx].grid(True) # Uncomment this line to show grid lines
#             title = f"{header}Layer {key}: {column} vs Raw"
#             if IS_FILTERED:
#                 title += " vs Reduced Noise"
#             axes[ax_idx].set_title(title)
#             ax_idx += 1  # Increment ax_idx

#     plt.tight_layout()
#     plt.show()
#     print("------------------")